# Measuring Background Pixel Correlations

Let's build on the last tutorial and measure the correlation properties of the background pixels we've found.

In [2]:
import os
import fitsio
import yaml
import esutil
import numpy as np

import matplotlib.pyplot as plt

## What is the correlation anyways?

To get a handle on this, let's step back and talk about variance and covariance. Suppose we have two quantities, `A` and `B`. Formally in statistics, these would be called random variables, but for our purposes let's not worry about the jargon. We are casually familiar with the computing the mean. However, the formal definition of the mean is an integral like this

$$
E[A] = \int dA\, A P(A)
$$

The usual definition of the mean

$$
\langle a\rangle = \frac{1}{N}\sum a_i
$$

is known as an *estimator* of the mean. (Here the `a_i` are draws from the distribution $P(A)$. For example, if $P(A)$ is a Gaussian distribution, then the `a_i` would be a set of numbers whose histogram approaches the shape of a Gaussian as we increase the size of the set.) An estimator is something we compute from the data we observe as opposed to something defined by $P(A)$ like $E[A]$ above. It turns out that our estimator for the mean is unbiased since 

$$
E[\langle a \rangle] = \int da_0 ... \int da_N\, \frac{1}{N}\sum a_i P(a_0) ... P(a_N) = \frac{1}{N}\sum E[A] = E[A]
$$

However, this is not a general property of estimators. 

Now that we've define the mean, let's define the variance. This quantity is 

$$
Var[A] = E[(A - E[A])^2] = \int dA\, (A - E[A])^2 P(A)
$$

The standard deviation is simply $Std[A] = \sqrt{Var[A]}$. There are again estimators for the standard deviation. You are likely familiar with the standard one with a factor of N-1 in the denominator. We won't go through the math here, but this odd looking N-1 is needed to make the estimator an unbiased estimate of the standard deviation.

Finally, we can define the covariance

$$
Cov[A, B] = E[(A-E[A])(B-E[B])] = \int dAdB\, (A-E[A])(B-E[B]) P(A, B) 
$$

where now this is a double integral and we have the joint distribution of $A$ and $B$, $P(A,B)$.

### Exercise 1

Compute $Cov[A,A]$. Do you recognize this quantity? What is the general property or rule here?

## Correlation, finally.

Now that we understand the covariance, the correlation is defined as

$$
Corr[A, B] = \frac{Cov[A,B]}{Std[A]\,Std[B]}
$$

The correlation has a nice property that it is always bounded between between -1 and 1. Positive values are indicate correlation (i.e., as A goes up, B goes up too) and negative values indicate anticorrelation (i.e., as A goes up, B goes down).

## Measuring the correlation of pixels in an image

For an image, the concept of correlation is a bit more complicated. Instead of two sets of values A and B (e.g., height and age or something), we a full image worth of pixels. Further, we might have more than one image to handle. To handle these issues, we need to make some definitions. First, we're going to measure the correlation of pixels separated by some distance, say one pixel. The idea here is that we are asking the question, if a given pixel goes up, do the pixels next to it go up or down? (By up and down here I really mean bigger or less than the mean.)

In math, we have for pixel value $V_ij$ and a separation of one pixel spacing in $j$ a quantity like

$$
C_{i,j+1} = \langle Cov[V_{i,j}, V_{i,j+1}]\rangle
$$

where the average is over all possible values of $i,j$. We can of course measure this for any offset in $\Delta i,\Delta j$. It is helpful to arrange these values into a 3x3 matrix where the center element is the above quantity with the offsets at zero and then each outer element is the quantity above with the offset given by the offset to the location in the matrix. Visually we have a grid of values like this

$$
\begin{array}{ccc}
C_{i-1,j+1} & C_{i,j+1} & C_{i+1,j+1}\\
C_{i-1,j} & C_{i,j} & C_{i+1,j}\\
C_{i-1,j-1} & C_{i,j-1} & C_{i+1,j-1}\\
\end{array}
$$

Finally, in our case, we have pixels which we want to ignore and so we cannot simply use all of the pixels we find. I have put a function below that does the computation we'd like.

In [11]:
import numba

@numba.njit()
def _meas_cov(im, msk, cov, nrm):
    for ic in range(im.shape[0]):
        io_min = max(ic-1, 0)
        io_max = min(ic+1, im.shape[0]-1) + 1
        
        for jc in range(im.shape[1]):
            jo_min = max(jc-1, 0)
            jo_max = min(jc+1, im.shape[1]-1) + 1
            
            if not msk[ic, jc]:
                continue
            
            for io in range(io_min, io_max):
                i = io - ic + 1
                
                for jo in range(jo_min, jo_max):
                    j = jo - jc + 1
                    
                    if not msk[io, jo]:
                        continue

                    cov[i, j] += (im[ic, jc] * im[io, jo])
                    nrm[i, j] += 1
                    
    for i in range(3):
        for j in range(3):
            cov[i, j] /= nrm[i, j]


def meas_cov(im, msk):
    """Measure the one-offset covariance of pixels in an image, ignoring bad ones.
    
    Parameters
    ----------
    im : np.ndarray
        The image.
    msk : np.ndarray
        The mask where True is a good pixel and False is a bad pixel.
    
    Returns
    -------
    cov : np.ndarray
        The 3x3 matrix of covariances between the pixels.
    """
    cov = np.zeros((3, 3))
    nrm = np.zeros((3, 3))
    mn = np.mean(im[msk])
    
    _meas_cov(im.astype(np.float64)-mn, msk, cov, nrm)
    return cov

### Exercise 2: Measure covariance matrix of Gaussian random image

1. Make an image of Gaussian random draws and measure the covariance matrix. I have put some code below to get you started.
2. Try different sizes (e.g., 10x10, 100x100, 1000x1000). What do you notice about the results in relationship to the correct answer as you increase the image size? What is the correct answer?

In [24]:
im = np.random.normal(size=(1000, 1000))
msk = np.ones_like(im)

# use the function above

## Exercise 3: Measure the covariance matrix for a single epoch image

Now let's move on to a single epoch image. I have put a bunch of code below to help in making the cuts etc.

In [25]:
def read_wcs(pth, ext=0):
    hdr = fitsio.read_header(pth, ext=ext)
    dct = {}
    for k in hdr.keys():
        try:
            dct[k.lower()] = hdr[k]
        except Exception:
            pass
    return esutil.wcsutil.WCS(dct)

In [26]:
meds_dir = "/cosmo/scratch/mrbecker/MEDS_DIR"
tilename = "DES0124-3332"
band = "i"
yaml_pth = os.path.join(
    meds_dir, 
    "des-pizza-slices-y6-v8/pizza_cutter_info/%s_%s_pizza_cutter_info.yaml" % (
        tilename, band
    )
)

with open(yaml_pth, "r") as fp:
    info = yaml.safe_load(fp.read())

In [32]:
coadd_wcs = read_wcs(info['image_path'], ext=info['image_ext'])
coadd_image = fitsio.read(info['image_path'], ext=info['image_ext'])
coadd_weight = fitsio.read(info['weight_path'], ext=info['weight_ext'])
coadd_bmask = fitsio.read(info['bmask_path'], ext=info['bmask_ext'])
coadd_seg = fitsio.read(info['seg_path'], ext=info['seg_ext'])

In [48]:
def get_mask_and_image(si, coadd_wcs, coadd_weight, coadd_bmask, coadd_seg):
    # read all of the data here
    se_wcs = read_wcs(si['image_path'], ext=si['image_ext'])
    se_image = fitsio.read(si['image_path'], ext=si['image_ext'])
    se_weight = fitsio.read(si['weight_path'], ext=si['weight_ext'])
    se_bmask = fitsio.read(si['bmask_path'], ext=si['bmask_ext'])
    se_bkg = fitsio.read(si['bkg_path'], ext=si['bkg_ext'])
    se_image -= se_bkg
    se_image *= si['scale']
    
    # handle the WCS transforms
    xind, yind = np.meshgrid(np.arange(se_image.shape[1]), np.arange(se_image.shape[0]))
    xind = xind.ravel()
    yind = yind.ravel()

    # map to ra, dec
    ra, dec = se_wcs.image2sky(x=xind+1, y=yind+1)

    # now map back to coadd pixel coordinates
    x_c, y_c = coadd_wcs.sky2image(ra, dec)

    # now take nearest pixel
    x_c = (x_c + 0.5).astype(np.int32)
    y_c = (y_c + 0.5).astype(np.int32)

    # finally, to get back to the indices into a coadd image, we need to subtract 
    # one per our discussion above
    x_i = x_c - 1
    y_i = y_c - 1
    
    # record unphysical rows and cols
    pmsk = (
        (x_i >= 0)
        & (x_i < 10000)
        & (y_i >= 0)
        & (y_i < 10000)
    ).reshape(se_image.shape)
    
    # now clip to proper range
    x_i = np.clip(x_i, 0, 9999)
    y_i = np.clip(y_i, 0, 9999)
    
    # now grab values
    se_coadd_seg = coadd_seg[y_i, x_i].reshape(se_image.shape)
    se_coadd_bmask = coadd_bmask[y_i, x_i].reshape(se_image.shape)
    se_coadd_weight = coadd_weight[y_i, x_i].reshape(se_image.shape)    
    
    
    msk = (
        (se_coadd_seg == 0)      # selects sky pixels
        & (se_coadd_bmask == 0)  # selects pixels without defects
        & (se_bmask == 0)
        & (se_coadd_weight > 0)  # selects pixels with finite variance
        & (se_weight > 0)
        & pmsk                   # select SE pixels that actuall fell onto the coadd image
    )    
    
    return msk, se_image

Your task is loop through all of the SE images for the coadd tile (each entry in `info['src_info']`) and measure the covariance matrices. Scale each one by the variance to compute the correlation matrix. Record them in a list. Then come up with some way to display the results in a plot that shows how different they are.

The code snippet below will measure the covariance matrix for a single entry of the list.

In [50]:
se_ind = 5  # 5th image is index 4
si = info['src_info'][se_ind]

msk, im = get_image_mask(si, coadd_wcs, coadd_weight, coadd_bmask, coadd_seg)
cov = meas_cov(im, msk)
corr = cov / cov[1, 1]
print(corr)

[[0.00274254 0.00329531 0.00263506]
 [0.00277856 1.         0.00277856]
 [0.00263506 0.00329531 0.00274254]]


In [ ]:
# here is the skeleton of a loop to get you started
results = []
for i in range(len(info['src_info'])):
    si = info['src_info'][i]

    results.append(corr)

## Save the Data

You can use python pickle to save the data for now. If your data is in the python variable `results`, then this looks like

In [ ]:
import pickle

# write the data
with open("test.pkl", "wb") as fp:
    pickle.dump(results, fp)

# read the data
with open("test.pkl", "rb") as fp:
    new_res = pickle.load(fp)